In [ ]:
import sys
import os

# Add the `src` directory to Python's module search path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Go up one level from `notebooks`
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import gtfs_kit as gk
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional, Any
from datetime import datetime, timedelta
from abc import ABC, abstractmethod
import time
from shapely.geometry import Point, Polygon 

from transit_opt.preprocessing.prepare_gtfs import GTFSDataPreparator


In [2]:
# Configure logging 
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
# Configure logging for your application
import logging
logging.basicConfig(level=logging.INFO)

# Use with different log levels
preparator = GTFSDataPreparator(
    gtfs_path='../data/external/study_area_gtfs_bus.zip',
    interval_hours=3,  # 8 periods per day
    date=None,  # Use full GTFS feed
    turnaround_buffer=1.15,  # 15% buffer
    max_round_trip_minutes= 240.0,  # Maximum round-trip time in minutes
    no_service_threshold_minutes= 480.0,  # Threshold for no-service mapping
    log_level="DEBUG"  # Use "INFO" for production, "DEBUG" for development
)

allowed_headways = [10, 15, 30, 60, 120]
opt_data = preparator.extract_optimization_data(allowed_headways)


2025-08-22 16:18:40,496 - transit_opt.preprocessing.prepare_gtfs - INFO - Initializing GTFSDataPreparator with 3h intervals
INFO:transit_opt.preprocessing.prepare_gtfs:Initializing GTFSDataPreparator with 3h intervals
2025-08-22 16:18:40,498 - transit_opt.preprocessing.prepare_gtfs - DEBUG - Configuration: turnaround_buffer=1.15, max_round_trip=240.0min, default_round_trip=60.0min
DEBUG:transit_opt.preprocessing.prepare_gtfs:Configuration: turnaround_buffer=1.15, max_round_trip=240.0min, default_round_trip=60.0min
2025-08-22 16:18:40,499 - transit_opt.preprocessing.prepare_gtfs - INFO - Loading GTFS feed from ../data/external/study_area_gtfs_bus.zip
INFO:transit_opt.preprocessing.prepare_gtfs:Loading GTFS feed from ../data/external/study_area_gtfs_bus.zip
2025-08-22 16:18:42,717 - transit_opt.preprocessing.prepare_gtfs - DEBUG - GTFS feed loaded successfully: 187 routes, 13974 trips
DEBUG:transit_opt.preprocessing.prepare_gtfs:GTFS feed loaded successfully: 187 routes, 13974 trips
2025

In [ ]:
opt_data['initial_solution'][1:20,]

In [ ]:
# =============================================================================
# STREAMLINED GTFS DATA PREPARATOR FOR DISCRETE OPTIMIZATION
# =============================================================================

class GTFSDataPreparator:
    """
    Streamlined GTFS data extraction focused on optimization essentials.
    
    Extracts only what's needed:
    - Headways by interval (current values from GTFS)
    - Round-trip times for vehicle constraints
    
    Supports discrete headway optimization where user specifies allowed values.
    """
    
    def __init__(self, 
                 gtfs_path: str,
                 interval_hours: int,
                 date: Optional[str] = None,
                 turnaround_buffer: float = 1.15,
                 default_round_trip_time: float = 60.0,
                 max_round_trip_minutes: float = 240.0):
        """
        Initialize GTFS data preparator.
        
        Args:
            gtfs_path: Path to GTFS ZIP file or directory
            interval_hours: Time interval duration in HOURS (must divide 24 evenly)
            date: Optional service date filter (YYYYMMDD format)
            turnaround_buffer: Round-trip time multiplier (1.15 = 15% buffer)
            default_round_trip_time: Fallback round-trip time in MINUTES
            max_round_trip_minutes: Maximum allowed round-trip time in MINUTES (GTFS may have regional trips longer than this)

        """
        # Input validation
        if 24 % interval_hours != 0:
            raise ValueError(f"interval_hours ({interval_hours}) must divide 24 evenly. "
                           f"Valid values: 1, 2, 3, 4, 6, 8, 12, 24")
            
        # Store configuration
        self.gtfs_path = gtfs_path
        self.date = date
        self.interval_hours = interval_hours
        self.n_intervals = 24 // interval_hours
        self.turnaround_buffer = turnaround_buffer
        self.default_round_trip_time = default_round_trip_time
        self.max_round_trip_minutes = max_round_trip_minutes
        
        # Load and cache GTFS data
        self._load_gtfs()
    
    def _load_gtfs(self) -> None:
        """Load GTFS feed and cache for optimization and reconstruction."""
        print(f"⏱️  Loading GTFS feed from {self.gtfs_path}...")
        start_time = time.time()
        
        # Load original feed (keep for reconstruction)
        self.feed = gk.read_feed(self.gtfs_path, dist_units='km')
        
        # Apply date filtering if specified
        if self.date:
            print(f"📅 Filtering GTFS for date: {self.date}")
            try:
                self.feed = gk.filter_feed_by_dates(self.feed, [self.date])
                print(f"   ✅ Filtered to {len(self.feed.trips)} trips for date {self.date}")
            except Exception as e:
                print(f"   ⚠️  Date filtering failed: {e}, using full feed")
        else:
            print("📅 Using full GTFS feed (all service periods)")
        
        # Cache tables for processing
        self.trips_df = self.feed.trips.copy()
        self.stop_times_df = self.feed.stop_times.copy()
        self.routes_df = self.feed.routes.copy()
        
        # Convert times to seconds for calculations
        self.stop_times_df['departure_seconds'] = self.stop_times_df['departure_time'].apply(
            self._safe_timestr_to_seconds
        )
        self.stop_times_df['arrival_seconds'] = self.stop_times_df['arrival_time'].apply(
            self._safe_timestr_to_seconds
        )
        
        load_time = time.time() - start_time
        print(f"✅ GTFS loaded and cached in {load_time:.2f} seconds")
        print(f"   📊 {len(self.trips_df):,} trips, {len(self.stop_times_df):,} stop times")
    
    def _safe_timestr_to_seconds(self, time_value: Any) -> float:
        """Safely convert GTFS time values to seconds from midnight."""
        try:
            if pd.isna(time_value):
                return np.nan
            if isinstance(time_value, str):
                return gk.helpers.timestr_to_seconds(time_value)
            else:
                return float(time_value)
        except Exception:
            return np.nan
    
    def extract_optimization_data(self, allowed_headways: List[float]) -> Dict[str, Any]:
        """IMPROVED: Extract data optimized for downstream algorithms."""
        
        # Extract route data first
        route_data = self._extract_route_essentials()
        n_routes = len(route_data)
        
        # Create headway mappings
        allowed_values = np.array(allowed_headways + [9999.0], dtype=np.float64)
        headway_to_index = {float(h): i for i, h in enumerate(allowed_values)}
        no_service_index = len(allowed_values) - 1
        
        # Create aligned arrays
        route_ids = [r['service_id'] for r in route_data]
        round_trip_times = np.array([r['round_trip_time'] for r in route_data], dtype=np.float64)
        current_headways = np.array([r['headways_by_interval'] for r in route_data], dtype=np.float64)
        
        # Create initial solution matrix
        initial_solution = self._create_initial_solution(current_headways, headway_to_index)
        
        # Build optimized structure
        return {
            'problem_type': 'discrete_headway_optimization',
            'n_routes': n_routes,
            'n_intervals': self.n_intervals,
            'n_choices': len(allowed_values),
            
            'decision_matrix_shape': (n_routes, self.n_intervals),
            'variable_bounds': (0, len(allowed_values)-1),
            'initial_solution': initial_solution,
            
            'allowed_headways': allowed_values,
            'headway_to_index': headway_to_index,
            'no_service_index': no_service_index,
            
            'routes': {
                'ids': route_ids,
                'round_trip_times': round_trip_times,
                'current_headways': current_headways,
            },
            
            'constraints': {
                'fleet_data': {
                    'round_trip_times': round_trip_times,  # Reference, not copy
                    'min_fleet_factor': 0.8,
                },
                'service_coverage': {
                    'min_service_ratio': 0.4,
                }
            },
            
            'intervals': {
                'labels': [f"{i*self.interval_hours:02d}-{(i+1)*self.interval_hours:02d}h" 
                        for i in range(self.n_intervals)],
                'hours': [(i*self.interval_hours, (i+1)*self.interval_hours) 
                        for i in range(self.n_intervals)],
                'duration_minutes': self.interval_hours * 60,
            },
            
            'metadata': {
                'gtfs_source': self.gtfs_path,
                'date_filter': self.date,
                'creation_timestamp': datetime.now().isoformat(),
                'filter_stats': {
                    'final_routes': n_routes,
                }
            },
            
            'reconstruction': {
                'gtfs_feed': self.feed,
                'route_mapping': {route_id: i for i, route_id in enumerate(route_ids)},
            }
        }
    
    def _extract_route_essentials(self) -> List[Dict[str, Any]]:
        """Extract only essential data: headways and round-trip times."""
        print(f"⏱️  Extracting route essentials with {self.interval_hours}-hour intervals...")
        
        all_services = self.trips_df['service_id'].unique()
        route_data = []

        # Keep track of number of routes filtered out because they exceed max_round_trip_minutes
        filtered_count = 0
        
        for service_id in all_services:
            service_trips = self.trips_df[self.trips_df['service_id'] == service_id]
            
            if len(service_trips) == 0:
                continue
            
            # Calculate headways by interval
            headways_by_interval = self._calculate_service_headways(service_id, service_trips)
            
            # Skip if no service found
            if np.all(np.isnan(headways_by_interval)):
                continue
            
            # Calculate round-trip time
            round_trip_time = self._calculate_round_trip_time(service_id, service_trips)

            # Filter out services with excessive round-trip times
            if round_trip_time > self.max_round_trip_minutes:
                print(f"   ⚠️  Filtered out route {service_id}: {round_trip_time:.1f} min round-trip (>{self.max_round_trip_minutes} min)")
                filtered_count += 1
                continue
            
            route_data.append({
                'service_id': service_id,
                'headways_by_interval': headways_by_interval,
                'round_trip_time': round_trip_time
            })
        
        print(f"✅ Extracted {len(route_data)} routes")
        return route_data
    
    def _calculate_service_headways(self, service_id: str, service_trips: pd.DataFrame) -> np.ndarray:
        """Calculate headway values for each time interval."""
        headways = np.full(self.n_intervals, np.nan)
        
        try:
            trip_ids = service_trips['trip_id'].tolist()
            service_stop_times = self.stop_times_df[
                self.stop_times_df['trip_id'].isin(trip_ids)
            ].copy()
            
            if len(service_stop_times) == 0:
                return headways
            
            # Get first departure for each trip
            first_departures = service_stop_times.loc[
                service_stop_times.groupby('trip_id')['stop_sequence'].idxmin()
            ][['trip_id', 'departure_seconds']].copy()
            
            first_departures['departure_hour'] = (first_departures['departure_seconds'] // 3600) % 24
            first_departures = first_departures.dropna()
            
            # Calculate headways for each interval
            for interval in range(self.n_intervals):
                start_hour = interval * self.interval_hours
                end_hour = (interval + 1) * self.interval_hours
                
                interval_departures = first_departures[
                    (first_departures['departure_hour'] >= start_hour) &
                    (first_departures['departure_hour'] < end_hour)
                ]['departure_seconds'].values
                
                if len(interval_departures) >= 2:
                    # Calculate average interval between departures
                    interval_departures = np.sort(interval_departures)
                    intervals = np.diff(interval_departures) / 60  # Convert to minutes
                    valid_intervals = intervals[intervals > 0]
                    if len(valid_intervals) > 0:
                        headways[interval] = np.mean(valid_intervals)
                elif len(interval_departures) == 1:
                    # Single trip - once per day service
                    headways[interval] = 24 * 60  # 1440 minutes
            
            return headways
            
        except Exception:
            return headways
    
    def _calculate_round_trip_time(self, service_id: str, service_trips: pd.DataFrame) -> float:
        """Calculate round-trip time with turnaround buffer."""
        try:
            trip_ids = service_trips['trip_id'].tolist()
            service_stop_times = self.stop_times_df[
                self.stop_times_df['trip_id'].isin(trip_ids)
            ].copy()
            
            if len(service_stop_times) == 0:
                return self.default_round_trip_time
            
            trip_durations = []
            for trip_id, trip_stops in service_stop_times.groupby('trip_id'):
                if len(trip_stops) >= 2:
                    trip_stops = trip_stops.sort_values('stop_sequence')
                    first_departure = trip_stops.iloc[0]['departure_seconds']
                    last_arrival = trip_stops.iloc[-1]['arrival_seconds']
                    
                    if pd.notna(first_departure) and pd.notna(last_arrival):
                        duration_minutes = (last_arrival - first_departure) / 60.0
                        if duration_minutes > 0:
                            trip_durations.append(duration_minutes)
            
            if trip_durations:
                median_one_way = np.median(trip_durations)
                return median_one_way * 2.0 * self.turnaround_buffer
            else:
                return self.default_round_trip_time
                
        except Exception:
            return self.default_round_trip_time
        
    def _create_initial_solution(self, current_headways: np.ndarray, headway_to_index: Dict[float, int]) -> np.ndarray:
        """Create initial solution matrix by mapping current headways to allowed indices."""
        n_routes, n_intervals = current_headways.shape
        initial_solution = np.zeros((n_routes, n_intervals), dtype=int)
        
        allowed_headway_values = list(headway_to_index.keys())[:-1]  # Exclude 9999
        no_service_index = headway_to_index[9999.0]
        
        for i in range(n_routes):
            for j in range(n_intervals):
                current_headway = current_headways[i, j]
                
                if np.isnan(current_headway):
                    # No service - use no_service_index
                    initial_solution[i, j] = no_service_index
                else:
                    # Find nearest allowed headway
                    distances = [abs(current_headway - h) for h in allowed_headway_values]
                    best_idx = np.argmin(distances)
                    initial_solution[i, j] = best_idx
        
        return initial_solution




In [ ]:
# =============================================================================
# FIXED GTFS RECONSTRUCTOR
# =============================================================================

class SimplifiedGTFSReconstructor:
    """
    UPDATED: Reconstructor compatible with improved optimization data structure.
    """
    
    def __init__(self, optimization_data: Dict[str, Any], optimization_result: Dict[str, Any]):
        self.optimization_data = optimization_data
        self.optimization_result = optimization_result
        
        # UPDATED: Access GTFS feed from new structure
        self.feed = optimization_data['reconstruction']['gtfs_feed']
        
        # Decode solution
        self.optimized_headways = self._decode_headway_solution()
    
    def _decode_headway_solution(self) -> np.ndarray:
        """Convert optimization solution indices to actual headway values."""
        solution_indices = self.optimization_result['headway_solution']
        allowed_headways = self.optimization_data['allowed_headways']
        no_service_index = self.optimization_data['no_service_index']
        
        n_routes, n_intervals = solution_indices.shape
        headways = np.full((n_routes, n_intervals), np.nan)
        
        for i in range(n_routes):
            for j in range(n_intervals):
                choice_idx = solution_indices[i, j]
                headway_value = allowed_headways[choice_idx]
                
                if choice_idx == no_service_index or headway_value >= 9000:
                    headways[i, j] = np.nan  # No service
                else:
                    headways[i, j] = headway_value
        
        return headways
    
    def reconstruct_gtfs(self, use_frequencies: bool = False) -> Any:
        """Reconstruct GTFS with proper stop_times.txt."""
        print("=== RECONSTRUCTING GTFS WITH OPTIMIZED HEADWAYS ===")
        
        # Start with copy of original feed
        new_feed = self.feed.copy()
        
        # Generate new stop_times and trips
        new_stop_times, new_trips = self._generate_stop_times_and_trips()
        
        # Update feed
        new_feed.stop_times = new_stop_times
        new_feed.trips = new_trips
        
        # Handle frequencies (optional)
        if use_frequencies and len(new_trips) > 0:
            frequencies_df = self._create_frequencies_table(new_trips)
            if len(frequencies_df) > 0:
                new_feed.frequencies = frequencies_df
                print(f"   📊 Added {len(frequencies_df):,} frequency entries")
            else:
                new_feed.frequencies = None
                print(f"   ⚠️  No frequencies generated - skipping frequencies.txt")
        else:
            new_feed.frequencies = None
            print(f"   📊 Frequencies.txt disabled - using stop_times.txt only")
        
        print(f"✅ Reconstructed GTFS with stop_times.txt:")
        print(f"   📊 {len(new_trips):,} trips")
        print(f"   📊 {len(new_stop_times):,} stop times")
        
        return new_feed
    
    def _generate_stop_times_and_trips(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Generate both stop_times and trips tables with proper relationships."""
        new_stop_times_list = []
        new_trips_list = []
        trip_id_counter = 1
        
        # UPDATED: Use new data structure
        route_ids = self.optimization_data['routes']['ids']
        n_intervals = self.optimization_data['n_intervals']
        interval_hours = self.optimization_data['intervals']['duration_minutes'] // 60
        
        print(f"   🔄 Generating trips and stop_times for {len(route_ids)} routes")
        
        for route_idx, service_id in enumerate(route_ids):
            # Get original trips for this service
            original_trips = self.feed.trips[self.feed.trips['service_id'] == service_id]
            
            if len(original_trips) == 0:
                continue
            
            # Use first trip as template
            template_trip = original_trips.iloc[0]
            template_trip_id = template_trip['trip_id']
            
            # Get template stop_times
            template_stops = self.feed.stop_times[
                self.feed.stop_times['trip_id'] == template_trip_id
            ].sort_values('stop_sequence').copy()
            
            if len(template_stops) == 0:
                continue
            
            # Convert template times to seconds for calculations
            template_stops['departure_seconds'] = template_stops['departure_time'].apply(
                self._safe_timestr_to_seconds
            )
            template_stops['arrival_seconds'] = template_stops['arrival_time'].apply(
                self._safe_timestr_to_seconds
            )
            
            # Generate trips for each interval with service
            route_trips_generated = 0
            for interval_idx in range(n_intervals):
                headway = self.optimized_headways[route_idx, interval_idx]
                
                # Skip intervals with no service
                if np.isnan(headway):
                    continue
                
                # UPDATED: Use interval hours from data structure
                start_hour, end_hour = self.optimization_data['intervals']['hours'][interval_idx]
                interval_duration_minutes = end_hour * 60 - start_hour * 60
                
                # Calculate number of trips needed in this interval
                n_trips = max(1, int(interval_duration_minutes / headway))
                
                # Generate trips spaced by optimized headway
                for trip_num in range(n_trips):
                    # Calculate start time for this trip
                    trip_start_minutes = start_hour * 60 + (trip_num * headway)
                    
                    # Don't exceed interval boundary
                    if trip_start_minutes >= end_hour * 60:
                        break
                    
                    # Create new trip with unique ID
                    new_trip_id = f"opt_{service_id}_{interval_idx}_{trip_num}"
                    new_trip = template_trip.copy()
                    new_trip['trip_id'] = new_trip_id
                    
                    # Clear any block_id to avoid conflicts
                    if 'block_id' in new_trip:
                        new_trip['block_id'] = f"block_{trip_id_counter}"
                    
                    new_trips_list.append(new_trip)
                    
                    # Generate stop_times for this trip
                    trip_stop_times = self._create_trip_stop_times(
                        template_stops, new_trip_id, trip_start_minutes
                    )
                    
                    if trip_stop_times is not None:
                        new_stop_times_list.append(trip_stop_times)
                        route_trips_generated += 1
                    
                    trip_id_counter += 1
            
            if route_trips_generated > 0 and route_idx < 5:  # Log first few routes
                print(f"   📍 Route {route_idx} ({service_id}): Generated {route_trips_generated} trips")
        
        # Combine all data
        if new_trips_list and new_stop_times_list:
            new_trips = pd.DataFrame(new_trips_list).reset_index(drop=True)
            new_stop_times = pd.concat(new_stop_times_list, ignore_index=True)
            
            print(f"   ✅ Generated {len(new_trips):,} trips with {len(new_stop_times):,} stop times")
        else:
            # Create empty but valid DataFrames
            new_trips = self.feed.trips.iloc[0:0].copy()
            new_stop_times = self.feed.stop_times.iloc[0:0].copy()
            print(f"   ⚠️  No trips generated - all routes mapped to no service")
        
        return new_stop_times, new_trips
    
    def _create_trip_stop_times(self, template_stops: pd.DataFrame, 
                              new_trip_id: str, trip_start_minutes: float) -> Optional[pd.DataFrame]:
        """Create stop_times for a single trip based on template."""
        try:
            # Calculate time offset
            template_start_seconds = template_stops.iloc[0]['departure_seconds']
            if pd.isna(template_start_seconds):
                return None
            
            trip_start_seconds = trip_start_minutes * 60
            time_offset = trip_start_seconds - template_start_seconds
            
            # Create new stop_times
            new_stop_times = template_stops.copy()
            new_stop_times['trip_id'] = new_trip_id
            
            # Adjust all times
            new_stop_times['departure_seconds'] = template_stops['departure_seconds'] + time_offset
            new_stop_times['arrival_seconds'] = template_stops['arrival_seconds'] + time_offset
            
            # Convert back to GTFS time strings
            new_stop_times['departure_time'] = new_stop_times['departure_seconds'].apply(
                self._seconds_to_timestr
            )
            new_stop_times['arrival_time'] = new_stop_times['arrival_seconds'].apply(
                self._seconds_to_timestr
            )
            
            # Remove helper columns
            new_stop_times = new_stop_times.drop(['departure_seconds', 'arrival_seconds'], 
                                               axis=1, errors='ignore')
            
            return new_stop_times
            
        except Exception as e:
            print(f"   ⚠️  Failed to create stop_times for trip {new_trip_id}: {e}")
            return None
    
    def _create_frequencies_table(self, trips_df: pd.DataFrame) -> pd.DataFrame:
        """Create frequencies.txt that uses ACTUAL trip IDs from the new trips."""
        frequencies_list = []
        n_intervals = self.optimization_data['n_intervals']
        route_ids = self.optimization_data['routes']['ids']
        
        for route_idx, service_id in enumerate(route_ids):
            # Get trips that were actually generated for this service
            service_trips = trips_df[trips_df['service_id'] == service_id]
            
            if len(service_trips) == 0:
                continue
            
            # Create frequency entries for each interval that has service
            for interval_idx in range(n_intervals):
                headway = self.optimized_headways[route_idx, interval_idx]
                
                if np.isnan(headway):
                    continue
                
                # Find a trip that was actually generated for this interval
                interval_trips = service_trips[
                    service_trips['trip_id'].str.contains(f'_{interval_idx}_', na=False)
                ]
                
                if len(interval_trips) == 0:
                    continue
                
                # Use the first trip from this interval as the frequency template
                template_trip_id = interval_trips.iloc[0]['trip_id']
                
                # UPDATED: Get interval hours from data structure
                start_hour, end_hour = self.optimization_data['intervals']['hours'][interval_idx]
                
                frequency_entry = {
                    'trip_id': template_trip_id,
                    'start_time': f"{start_hour:02d}:00:00",
                    'end_time': f"{end_hour:02d}:00:00",
                    'headway_secs': int(headway * 60),
                    'exact_times': 0
                }
                
                frequencies_list.append(frequency_entry)
        
        return pd.DataFrame(frequencies_list)
    
    # Helper methods remain the same
    def _safe_timestr_to_seconds(self, time_value: Any) -> float:
        """Safely convert GTFS time strings to seconds."""
        try:
            if pd.isna(time_value):
                return np.nan
            if isinstance(time_value, str):
                return gk.helpers.timestr_to_seconds(time_value)
            else:
                return float(time_value)
        except Exception:
            return np.nan
    
    def _seconds_to_timestr(self, seconds: float) -> str:
        """Convert seconds to GTFS time string format."""
        if pd.isna(seconds):
            return "00:00:00"
        
        # Handle times > 24 hours (GTFS allows this)
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        
        return f"{hours:02d}:{minutes:02d}:{secs:02d}"

In [ ]:

# =============================================================================
# COMPLETE WORKFLOW: DATA PREPARATION → OPTIMIZATION → RECONSTRUCTION
# =============================================================================

# 1. PREPARE OPTIMIZATION DATA
print("=== STEP 1: PREPARING OPTIMIZATION DATA ===")
preparator = GTFSDataPreparator(
    gtfs_path='../data/external/study_area_gtfs_bus.zip',
    interval_hours=3,  # 8 periods per day
    date=None,  # Use full GTFS feed
    turnaround_buffer=1.15,  # 15% buffer
    max_round_trip_minutes= 240.0  # Maximum round-trip time in minutes
)

# Define allowed headway values for discrete optimization
allowed_headways = [5, 10, 15, 20, 30, 45, 60, 90, 120]  # minutes

# Extract optimization data
optimization_data = preparator.extract_optimization_data(allowed_headways)

# 2. SIMULATE OPTIMIZATION RESULT (since you don't have the actual optimizer yet)
print("\n=== STEP 2: SIMULATING OPTIMIZATION RESULT ===")
# For now, use initial solution as the "optimized" result
simulated_result = {
    'headway_solution': optimization_data['initial_solution'],
    'objective_value': 1000.0,  # Placeholder
    'solve_time': 5.0,  # Placeholder
    'status': 'optimal'
}

print(f"✅ Using initial solution as optimization result")
print(f"   📊 Solution shape: {simulated_result['headway_solution'].shape}")

# 3. RECONSTRUCT GTFS WITH OPTIMIZED HEADWAYS
print("\n=== STEP 3: RECONSTRUCTING GTFS ===")
reconstructor = SimplifiedGTFSReconstructor(optimization_data, simulated_result)

# Generate GTFS with stop_times.txt (required for all simulations)
new_gtfs_feed = reconstructor.reconstruct_gtfs(use_frequencies=False)


# 4. SAVE THE COMPLETE GTFS FEED
print("\n=== STEP 4: SAVING OPTIMIZED GTFS ===")
if len(new_gtfs_feed.trips) > 0:
    output_path = '../data/processed/optimized_gtfs.zip'
    
    # Ensure output directory exists
    import os
    os.makedirs('../data/processed', exist_ok=True)
    
    # Use gtfs-kit's to_file() method - it handles ZIP automatically
    new_gtfs_feed.to_file(output_path)
    print(f"✅ Complete GTFS with stop_times.txt saved to: {output_path}")
    
else:
    print("⚠️  No trips generated - check optimization solution")

print("\n=== WORKFLOW COMPLETE ===")

In [ ]:
simulated_result

In [ ]:
# print the optimisation_data initial_solution dictionary

optimization_data['initial_solution']  # Display the initial solution for debugging


In [ ]:
new_gtfs_feed.stop_times.head(50)  # Display the first few trips in the reconstructed GTFS feed

In [ ]:
# Add this new cell to your notebook
import numpy as np
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.core.variable import Real, Integer
import matplotlib.pyplot as plt

class SpatialEquityProblem(ElementwiseProblem):
    """
    PyMOO problem definition for spatial equity optimization.
    """
    
    def __init__(self, optimization_data, zone_system, min_fleet_factor=0.8):
        self.opt_data = optimization_data
        self.zone_system = zone_system
        self.min_fleet_factor = min_fleet_factor
        
        # Problem dimensions
        self.n_routes = optimization_data['n_routes']
        self.n_intervals = optimization_data['n_intervals'] 
        self.n_choices = optimization_data['n_choices']
        self.n_vars = self.n_routes * self.n_intervals
        
        # Calculate fleet constraints
        self.baseline_vehicles = self._calculate_baseline_fleet()
        self.min_required_vehicles = int(self.baseline_vehicles * min_fleet_factor)
        
        print(f"🚗 Fleet constraints:")
        print(f"   📊 Current fleet size: {self.baseline_vehicles} vehicles")
        print(f"   📊 Minimum required: {self.min_required_vehicles} vehicles ({min_fleet_factor:.1%})")
        
        # Define problem: discrete variables (0 to n_choices-1)
        super().__init__(
            n_var=self.n_vars,
            n_obj=1,  # Single objective: minimize variance
            n_ieq_constr=2,  # 2 inequality constraints
            vars={
                f'x{i}': Integer(bounds=(0, self.n_choices-1)) 
                for i in range(self.n_vars)
            }
        )
    
    def _calculate_baseline_fleet(self):
        """Calculate current fleet size."""
        initial_solution = self.opt_data['initial_solution']
        vehicles_per_zone = self.zone_system.calculate_vehicles_per_zone(
            initial_solution, self.opt_data
        )
        return int(np.sum(vehicles_per_zone))
    
    def _evaluate(self, x, out, *args, **kwargs):
        """
        Evaluate single solution.
        
        Args:
            x: Decision variables (flattened array of discrete choices)
        """
        # Convert to solution matrix
        solution_matrix = x.reshape(self.n_routes, self.n_intervals)
        
        # Calculate vehicles per zone
        vehicles_per_zone = self.zone_system.calculate_vehicles_per_zone(
            solution_matrix, self.opt_data
        )
        
        # Primary objective: minimize variance in vehicles per zone
        if len(vehicles_per_zone) > 1 and np.sum(vehicles_per_zone) > 0:
            objective = np.var(vehicles_per_zone)
        else:
            objective = 0.0
        
        # Constraint 1: Minimum fleet size
        total_vehicles = np.sum(vehicles_per_zone)
        fleet_constraint = self.min_required_vehicles - total_vehicles  # <= 0
        
        # Constraint 2: Maximum service reduction (no more than 60% no-service)
        no_service_count = np.sum(solution_matrix == (self.n_choices - 1))
        no_service_ratio = no_service_count / solution_matrix.size
        service_constraint = no_service_ratio - 0.6  # <= 0
        
        # Set outputs
        out["F"] = [objective]  # Objective (minimize variance)
        out["G"] = [fleet_constraint, service_constraint]  # Constraints (≤ 0)

def run_pymoo_optimization(optimization_data, zone_system, min_fleet_factor=0.8):
    """
    Run spatial equity optimization using PyMOO PSO.
    """
    print("🚀 Starting PyMOO PSO Optimization...")
    
    # Create problem
    problem = SpatialEquityProblem(optimization_data, zone_system, min_fleet_factor)
    
    # Setup PSO algorithm
    algorithm = PSO(
        pop_size=30,           # Number of particles
        w=0.9,                 # Inertia weight
        c1=2.0,                # Cognitive parameter
        c2=2.0,                # Social parameter
        adaptive=True,         # Adaptive parameters
        max_velocity_rate=0.2  # Max velocity as fraction of variable range
    )
    
    # Run optimization
    result = minimize(
        problem,
        algorithm,
        termination=('n_gen', 100),  # 100 generations
        verbose=True,
        save_history=True
    )
    
    # Extract best solution
    best_x = result.X
    best_solution_matrix = best_x.reshape(
        optimization_data['n_routes'], 
        optimization_data['n_intervals']
    )
    
    print(f"✅ PyMOO PSO completed")
    print(f"   🎯 Final objective value: {result.F[0]:.4f}")
    print(f"   📊 Constraint violations: {result.G}")
    
    return {
        'headway_solution': best_solution_matrix,
        'objective_value': result.F[0],
        'constraints': result.G,
        'solve_time': 0.0,  # PyMOO doesn't track this directly
        'status': 'optimal',
        'pymoo_result': result  # Keep full result for analysis
    }

def plot_pymoo_convergence(result):
    """Plot PyMOO optimization convergence."""
    if not hasattr(result, 'history'):
        print("⚠️  No history available for plotting")
        return
    
    # Extract convergence data
    generations = []
    objectives = []
    
    for entry in result.history:
        generations.append(entry.n_gen)
        # Get best objective value in this generation
        best_f = np.min(entry.pop.get("F"))
        objectives.append(best_f)
    
    # Plot convergence
    plt.figure(figsize=(12, 5))
    
    # Objective convergence
    plt.subplot(1, 2, 1)
    plt.plot(generations, objectives, 'b-', linewidth=2)
    plt.xlabel('Generation')
    plt.ylabel('Best Objective (Variance)')
    plt.title('PyMOO PSO - Objective Convergence')
    plt.grid(True, alpha=0.3)
    
    # Constraint violations (if any)
    plt.subplot(1, 2, 2)
    if result.G is not None and len(result.G) > 0:
        constraint_labels = ['Fleet Size', 'Service Level']
        plt.bar(constraint_labels, result.G)
        plt.ylabel('Constraint Violation')
        plt.title('Final Constraint Status\n(≤ 0 = satisfied)')
        plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    else:
        plt.text(0.5, 0.5, 'No Constraints\nViolated', 
                ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('Constraint Status')
    
    plt.tight_layout()
    plt.show()

In [ ]:
class HexagonalZoneSystem:
    """
    Optimized hexagonal zoning system using spatial indexing.
    """
    
    def __init__(self, 
                 gtfs_feed,
                 hex_size_km: float = 2.0,
                 crs: str = "EPSG:4326"):
        self.gtfs_feed = gtfs_feed
        self.hex_size_km = hex_size_km
        self.crs = crs
        
        # Create stop locations GeoDataFrame
        self.stops_gdf = self._create_stops_geodataframe()
        
        # Generate hexagonal grid
        self.hex_grid = self._create_hexagonal_grid()
        
        # OPTIMIZED: Use spatial join instead of nested loops
        self.stop_zone_mapping = self._fast_map_stops_to_zones()
        
        # OPTIMIZED: Pre-compute route-stop mappings
        self._precompute_route_stop_mappings()
    
    def _fast_map_stops_to_zones(self) -> Dict[str, str]:
        """
        OPTIMIZED: Use spatial join - O(S + Z) instead of O(S × Z).
        """
        print("🚀 Using spatial join for zone mapping...")
        
        # Spatial join: finds containing zone for each stop in one operation
        stops_with_zones = gpd.sjoin(
            self.stops_gdf, 
            self.hex_grid, 
            how='left', 
            predicate='within'
        )
        
        # Convert to dictionary
        stop_zone_map = {}
        for idx, row in stops_with_zones.iterrows():
            if pd.notna(row['zone_id']):
                stop_zone_map[row['stop_id']] = row['zone_id']
            else:
                # Handle stops not in any zone (find nearest)
                stop_point = row.geometry
                distances = self.hex_grid.geometry.distance(stop_point)
                nearest_zone_idx = distances.idxmin()
                stop_zone_map[row['stop_id']] = self.hex_grid.loc[nearest_zone_idx, 'zone_id']
        
        return stop_zone_map
    
    def _precompute_route_stop_mappings(self):
        """
        OPTIMIZED: Pre-compute all route → stops mappings to avoid repeated filtering.
        """
        print("🚀 Pre-computing route-stop mappings...")
        
        self.route_stops_cache = {}
        
        # Group trips by service_id once
        trips_by_service = self.gtfs_feed.trips.groupby('service_id')['trip_id'].apply(list).to_dict()
        
        # Group stop_times by trip_id once  
        stop_times_by_trip = self.gtfs_feed.stop_times.groupby('trip_id')['stop_id'].apply(set)
        
        for service_id, trip_ids in trips_by_service.items():
            # Get all unique stops for this service
            service_stops = set()
            for trip_id in trip_ids:
                if trip_id in stop_times_by_trip:
                    service_stops.update(stop_times_by_trip[trip_id])
            
            self.route_stops_cache[service_id] = service_stops
    
    def calculate_vehicles_per_zone(self, 
                              solution_matrix: np.ndarray,
                              optimization_data: Dict[str, Any]) -> np.ndarray:
        """UPDATED: Use new optimization data structure."""
        # Initialize zone counts
        zone_counts = {zone_id: 0 for zone_id in self.hex_grid['zone_id']}
        
        # UPDATED: Access data from new structure
        route_ids = optimization_data['routes']['ids']
        allowed_headways = optimization_data['allowed_headways']
        round_trip_times = optimization_data['routes']['round_trip_times']
        no_service_index = optimization_data['no_service_index']
        
        for route_idx, service_id in enumerate(route_ids):
            # Use cached stops instead of filtering DataFrames
            if service_id not in self.route_stops_cache:
                continue
            
            service_stops = self.route_stops_cache[service_id]
            
            # Calculate max vehicles for this route
            max_vehicles = 0
            for interval_idx in range(optimization_data['n_intervals']):
                choice_idx = solution_matrix[route_idx, interval_idx]
                
                # Skip no-service choices
                if choice_idx == no_service_index:
                    continue
                    
                headway = allowed_headways[choice_idx]
                
                if headway < 9000:
                    round_trip = round_trip_times[route_idx]
                    vehicles_in_interval = max(1, int(round_trip / headway))
                    max_vehicles = max(max_vehicles, vehicles_in_interval)
            
            # Add vehicles to zones served by this route
            zones_served = {
                self.stop_zone_mapping[stop_id] 
                for stop_id in service_stops 
                if stop_id in self.stop_zone_mapping
            }
            
            for zone_id in zones_served:
                zone_counts[zone_id] += max_vehicles
        
        return np.array(list(zone_counts.values()))

    def _create_hexagonal_grid(self) -> gpd.GeoDataFrame:
        """
        IMPROVED: Use proper hexagonal grid (H3 library recommended).
        """
        # For now, keep your existing implementation
        # TODO: Replace with H3 hexagons for true hexagonal tiling
        bounds = self.stops_gdf.total_bounds
        
        buffer = self.hex_size_km / 111
        minx, miny, maxx, maxy = bounds
        minx -= buffer
        miny -= buffer  
        maxx += buffer
        maxy += buffer
        
        hex_polygons = []
        zone_ids = []
        
        x_steps = int((maxx - minx) / (self.hex_size_km / 111))
        y_steps = int((maxy - miny) / (self.hex_size_km / 111))
        
        zone_id = 0
        for i in range(x_steps):
            for j in range(y_steps):
                x = minx + i * (self.hex_size_km / 111)
                y = miny + j * (self.hex_size_km / 111)
                
                from shapely.geometry import Polygon
                cell = Polygon([
                    (x, y), (x + self.hex_size_km/111, y),
                    (x + self.hex_size_km/111, y + self.hex_size_km/111),
                    (x, y + self.hex_size_km/111)
                ])
                
                hex_polygons.append(cell)
                zone_ids.append(f"zone_{zone_id}")
                zone_id += 1
        
        return gpd.GeoDataFrame({
            'zone_id': zone_ids,
            'geometry': hex_polygons
        }, crs=self.crs)
    
    def _create_stops_geodataframe(self) -> gpd.GeoDataFrame:
        """Same as original - this is already efficient."""
        stops = self.gtfs_feed.stops.copy()
        geometry = [Point(lon, lat) for lon, lat in zip(stops['stop_lon'], stops['stop_lat'])]
        return gpd.GeoDataFrame(stops, geometry=geometry, crs=self.crs)

In [ ]:
# 1.5. CREATE ZONING SYSTEM
print("\n=== STEP 1.5: CREATING HEXAGONAL ZONING SYSTEM ===")
zone_system = HexagonalZoneSystem(
    gtfs_feed=optimization_data['reconstruction']['gtfs_feed'],
    hex_size_km=2.0  # 2km hexagons
)

print(f"✅ Created {len(zone_system.hex_grid)} hexagonal zones")

# After creating zone_system, add:
print(f"DEBUG: Created {len(zone_system.hex_grid)} zones")
print(f"DEBUG: Mapped {len(zone_system.stop_zone_mapping)} stops to zones")
print(f"DEBUG: Route cache has {len(zone_system.route_stops_cache)} routes")

# Sample some mappings:
sample_stops = list(zone_system.stop_zone_mapping.items())[:5]
print(f"DEBUG: Sample stop-zone mappings: {sample_stops}")

# 2. RUN PYMOO OPTIMIZATION  
print("\n=== STEP 2: PYMOO PSO SPATIAL EQUITY OPTIMIZATION ===")

# Add debugging before optimization
print("🔍 DEBUGGING OPTIMIZATION DATA:")

# FIXED: Use correct paths to access data
route_ids = optimization_data['routes']['ids']                    # ← Already correct
round_trip_times = optimization_data['routes']['round_trip_times'] # ← FIXED

print(f"Route IDs: {len(route_ids)} routes")
print(f"Round trip times: {len(round_trip_times)} values")

# Check for None/NaN in round trip times
none_count = sum(1 for x in round_trip_times if x is None)
nan_count = sum(1 for x in round_trip_times if pd.isna(x))
print(f"None round trip times: {none_count}")
print(f"NaN round trip times: {nan_count}")

# Check route cache
missing_routes = [rid for rid in route_ids if rid not in zone_system.route_stops_cache]
print(f"Missing routes in cache: {len(missing_routes)}")
if missing_routes:
    print(f"First few missing: {missing_routes[:5]}")

# Check stop mappings
print(f"Stop-zone mappings: {len(zone_system.stop_zone_mapping)}")

# Run PyMOO optimization
pymoo_result = run_pymoo_optimization(
    optimization_data, 
    zone_system, 
    min_fleet_factor=0.8
)

# Plot convergence
plot_pymoo_convergence(pymoo_result['pymoo_result'])

# Use PyMOO result for reconstruction
pso_result = pymoo_result  # Compatible format

In [ ]:
optimization_data

In [ ]:
# DEBUGGING CELL: Find the problematic data
print("🔍 DETAILED DEBUGGING:")

route_ids = optimization_data['route_ids']
round_trip_times = optimization_data['round_trip_times']

print(f"Data types:")
print(f"  round_trip_times type: {type(round_trip_times)}")
print(f"  First 5 values: {round_trip_times[:5]}")
print(f"  Value types: {[type(x) for x in round_trip_times[:5]]}")

# Check for extreme values
print(f"\nRound trip time statistics:")
print(f"  Min: {np.min(round_trip_times)}")
print(f"  Max: {np.max(round_trip_times)}")
print(f"  Mean: {np.mean(round_trip_times)}")

# Look for problematic values
extreme_values = [(i, rt) for i, rt in enumerate(round_trip_times) if rt > 200 or rt < 5]
if extreme_values:
    print(f"Extreme round trip times: {extreme_values[:10]}")

# Test a single vehicle calculation
test_headway = 15.0
test_round_trip = round_trip_times[0]
test_vehicles = test_round_trip / test_headway
print(f"\nTest calculation:")
print(f"  Round trip: {test_round_trip} (type: {type(test_round_trip)})")
print(f"  Headway: {test_headway}")
print(f"  Vehicles needed: {test_vehicles}")

In [ ]:
# DIAGNOSTIC: Find problematic routes
print("\n🔍 ANALYZING EXTREME ROUND-TRIP TIMES:")

extreme_indices = [i for i, rt in enumerate(round_trip_times) if rt > 200]
print(f"Found {len(extreme_indices)} routes with >200min round-trips")

for i in extreme_indices[:10]:  # Show first 10
    service_id = route_ids[i]
    rt_time = round_trip_times[i]
    
    # Get sample trip for this service
    service_trips = optimization_data['gtfs_feed'].trips[
        optimization_data['gtfs_feed'].trips['service_id'] == service_id
    ]
    
    if len(service_trips) > 0:
        sample_trip = service_trips.iloc[0]['trip_id']
        trip_stops = optimization_data['gtfs_feed'].stop_times[
            optimization_data['gtfs_feed'].stop_times['trip_id'] == sample_trip
        ].sort_values('stop_sequence')
        
        if len(trip_stops) >= 2:
            start_time = trip_stops.iloc[0]['departure_time']
            end_time = trip_stops.iloc[-1]['arrival_time']
            print(f"  Route {service_id}: {rt_time:.1f}min ({start_time} → {end_time})")